This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers
import numpy as np
from tensorflow.keras.preprocessing import image

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input


2025-11-19 18:11:42.319496: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-19 18:11:42.651404: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-19 18:11:44.635542: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
# Paths (mirrors the notebook)
train_dir = "/home/zim/Downloads/archive/train"
test_dir = "/home/zim/Downloads/archive/test"
  # e.g. './muffin-vs-chihuahua/test'

In [3]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10


In [4]:
# DATA PREPROCESSING & AUGMENTATION

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2,
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Recreate generators with updated IMG_SIZE
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training",
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation",
)



test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=False,
)

Found 3788 images belonging to 2 classes.


Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [5]:

# Build improved model
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True,
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Load pretrained ResNet50 without top layers
resnet_base = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
)


# Freeze the base so only our classifier layers train
resnet_base.trainable = False


# Build final model
model = models.Sequential([
    resnet_base,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.4),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.4),
    layers.Dense(1, activation="sigmoid")
])

2025-11-19 18:11:45.639073: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 77s 1us/step


In [6]:
# Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,850,113 (90.98 MB)

 Trainable params: 262,401 (1.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [7]:
# Train
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator
)

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 248s 2s/step - accuracy: 0.5145 - loss: 0.7994 - val_accuracy: 0.7354 - val_loss: 0.6531
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 242s 2s/step - accuracy: 0.5523 - loss: 0.7288 - val_accuracy: 0.7048 - val_loss: 0.6259
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 244s 2s/step - accuracy: 0.5950 - loss: 0.6774 - val_accuracy: 0.7058 - val_loss: 0.6223
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 244s 2s/step - accuracy: 0.6125 - loss: 0.6598 - val_accuracy: 0.7407 - val_loss: 0.6029
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 241s 2s/step - accuracy: 0.6193 - loss: 0.6449 - val_accuracy: 0.7513 - val_loss: 0.5923
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 237s 2s/step - accuracy: 0.6301 - loss: 0.6392 - val_accuracy: 0.7545 - val_loss: 0.5817
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 238s 2s/step - accuracy: 0.6460 - loss: 0.6281 - val_accuracy: 0.7556 - val_loss: 0.5736
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 241s 2s/step - accuracy: 0.6729 - loss: 0.6149 - val_accu

In [8]:

# Evaluate
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")


37/37 ━━━━━━━━━━━━━━━━━━━━ 62s 2s/step - accuracy: 0.7247 - loss: 0.5751
Test Accuracy: 0.7247


In [9]:

# Save with requested name
out_path = "exercise_6_resnet_model.h5"
model.save(out_path)
print(f"Model saved to {out_path}")

Model saved to exercise_6_resnet_model.h5


In [10]:

filenames = test_generator.filenames
if len(filenames) >= 2:
    sample1 = os.path.join(test_dir, filenames[0])
    sample2 = os.path.join(test_dir, filenames[1])
    print("Sample files:", sample1, sample2)

    def predict_file(fp):
        img = image.load_img(fp, target_size=IMG_SIZE)
        arr = image.img_to_array(img)
        arr = preprocess_input(arr)  # <-- IMPORTANT: ResNet preprocessing
        arr = np.expand_dims(arr, axis=0)

        pred = model.predict(arr)[0, 0]
        label = "Chihuahua" if pred >= 0.5 else "Muffin"
        return label, float(pred)

    l1, p1 = predict_file(sample1)
    l2, p2 = predict_file(sample2)
    print(f"Prediction 1: {l1} (confidence {p1:.4f})")
    print(f"Prediction 2: {l2} (confidence {p2:.4f})")
else:
    print("Not enough test images to run sample predictions")

Sample files: /home/zim/Downloads/archive/test/chihuahua/img_0_1071.jpg /home/zim/Downloads/archive/test/chihuahua/img_0_1074.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Prediction 1: Muffin (confidence 0.0000)
Prediction 2: Muffin (confidence 0.0000)
